In [99]:
import datetime
import pandas as pd
import numpy as np
print(datetime.datetime.now())

2022-09-03 22:26:42.174457


In [ ]:
import random
index = random.randint(1,3)
opcoes = ['SG','SP','SE']


In [ ]:
import requests
import random
import time
import tqdm


opcoes = ['SG','SP','SE']

for i in tqdm.tqdm(range(0,10)):
    index = random.randint(0,2)
    url = "http://localhost:8000/senha/" + opcoes[index]
    r = requests.get(url)
    time.sleep(index)

In [98]:
import requests
import random
import time
import tqdm


for i in range(0,10):
    guiche = random.randint(1,5)
    url = "http://localhost:8000/chamada/" + str(guiche)
    r = requests.get(url)
    print(r.json())
    time.sleep(2)

ConnectionError: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /chamada/5 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000200FD7CB460>: Failed to establish a new connection: [WinError 10061] Nenhuma conexão pôde ser feita porque a máquina de destino as recusou ativamente'))

In [100]:
df = pd.read_csv('atendimentos.csv')
df

,ID,Tipo_Senha,Numeracao,Codigo_Senha,Data_Emissao,Guiche,Data_Atendimento
0,1,SP,1,SP001,2022-09-03 21:59:08.066693,1.0,2022-09-03 22:10:23.128442
1,2,SP,2,SP002,2022-09-03 21:59:11.122359,1.0,2022-09-03 22:10:33.998144
2,3,SE,1,SE001,2022-09-03 21:59:14.183902,2.0,2022-09-03 22:10:30.177229
3,4,SP,3,SP003,2022-09-03 21:59:18.257014,4.0,2022-09-03 22:10:43.407901
4,5,SP,4,SP004,2022-09-03 21:59:21.318013,3.0,2022-09-03 22:11:02.347897
5,6,SG,1,SG001,2022-09-03 21:59:24.376220,3.0,2022-09-03 22:10:36.465818
6,7,SG,2,SG002,2022-09-03 21:59:26.436546,1.0,2022-09-03 22:10:51.522244
7,8,SG,3,SG003,2022-09-03 21:59:28.484014,NaN,NaN
8,9,SG,4,SG004,2022-09-03 21:59:30.534508,NaN,NaN
9,10,SG,5,SG005,2022-09-03 21:59:32.586410,NaN,NaN


In [106]:
df = pd.read_csv("atendimentos.csv")
max_id = df.ID.max()
if np.isnan(df.ID.max()):
    max_id = 0

df2 = pd.DataFrame(
    [[
        max_id + 1
        ,'SP'
        ,1
        ,'codigo_senha'
        , datetime.datetime.now()
        ]]
    , columns=['ID','Tipo_Senha','Numeracao','Codigo_Senha','Data_Emissao'])

df = pd.concat([df,df2]).reset_index(drop=True)

,ID,Tipo_Senha,Numeracao,Codigo_Senha,Data_Emissao,Guiche,Data_Atendimento
0,1,SP,1,SP001,2022-09-03 21:59:08.066693,1.0,2022-09-03 22:10:23.128442
1,2,SP,2,SP002,2022-09-03 21:59:11.122359,1.0,2022-09-03 22:10:33.998144
2,3,SE,1,SE001,2022-09-03 21:59:14.183902,2.0,2022-09-03 22:10:30.177229
3,4,SP,3,SP003,2022-09-03 21:59:18.257014,4.0,2022-09-03 22:10:43.407901
4,5,SP,4,SP004,2022-09-03 21:59:21.318013,3.0,2022-09-03 22:11:02.347897
5,6,SG,1,SG001,2022-09-03 21:59:24.376220,3.0,2022-09-03 22:10:36.465818
6,7,SG,2,SG002,2022-09-03 21:59:26.436546,1.0,2022-09-03 22:10:51.522244
7,8,SG,3,SG003,2022-09-03 21:59:28.484014,NaN,NaN
8,9,SG,4,SG004,2022-09-03 21:59:30.534508,NaN,NaN
9,10,SG,5,SG005,2022-09-03 21:59:32.586410,NaN,NaN


In [ ]:
df[~df["Data_Atendimento"].isna()].sort_values(by = "Data_Atendimento").tail(1)

In [ ]:
df.loc[df.ID==8,"Guiche"] = 1

In [ ]:
df.loc[df.ID==8,"Data_Atendimento"] = datetime.datetime.now()

In [ ]:
df

In [ ]:
df[df.ID == 8]

In [ ]:
ultimo_chamado = df[~df["Data_Atendimento"].isna()].sort_values(by = "Data_Atendimento").tail(1)
if len(ultimo_chamado) == 0:
    prioridade = ['SP','SE','SG']
else:
    ultimo_tipo_senha = ultimo_chamado["Tipo_Senha"].values[0]
    if (ultimo_tipo_senha == 'SP'):
        prioridade = ['SE','SG','SP']
    else:
        prioridade = ['SP','SE','SG']

### Checar se tem algum da prioridade
prioridade

In [ ]:
chamados_esperando = df[df["Data_Atendimento"].isna()].sort_values(by = "Data_Emissao")
for tipo in prioridade:
    chamados_prioritarios = chamados_esperando[chamados_esperando["Tipo_Senha"] == tipo].head(1)
    if (len(chamados_prioritarios) != 0):
        id_chamado = chamados_prioritarios.ID.values[0]
        break

id_chamado

In [ ]:
chamados_esperando = df[df["Data_Atendimento"].isna()].sort_values(by = "Data_Emissao")
chamados_prioritarios = chamados_esperando[chamados_esperando["Tipo_Senha"] == prioridade]
if (len(chamados_prioritarios) != 0):
    proximo = chamados_prioritarios.head(1)

In [ ]:
if ultimo_tipo_senha == "SP":
    

In [ ]:
ultimo_tipo_senha

In [ ]:
ultimo_tipo_senha

In [ ]:
### Análise se o pedido de senha foi feito fora do horário do expediente
inicio_expediente = pd.to_datetime(datetime.datetime.now().replace(hour = 7, minute = 0, second = 0, microsecond = 0))
fim_expediente = pd.to_datetime(datetime.datetime.now().replace(hour = 17, minute = 0, second = 0, microsecond = 0))

# horario_atual = pd.to_datetime(datetime.datetime.now())
horario_atual = pd.to_datetime(datetime.datetime.now().replace(hour = 6))

if (horario_atual < inicio_expediente) or (horario_atual > fim_expediente):
    print({"senha": "Fora do Expediênte de Trabalho"})

In [ ]:
print(a)

In [ ]:
horario_atual

In [ ]:
inicio_expediente

In [ ]:
def ultima_senha(tipo, inicio_expediente):
    df = pd.read_csv("atendimentos.csv")

    ultimo_registro = df[df["Tipo_Senha"] == tipo].tail(1)
    data_ultimo_registro = pd.to_datetime(ultimo_registro["Data_Emissao"]).values[0]

    if data_ultimo_registro < inicio_expediente:
        last_password = 0
    else:
        last_password = int(ultimo_registro["Numeracao"])

    return last_password

In [ ]:
df

In [ ]:
df = df.append({
            'ID' : max_id + 1
            ,'Tipo_Senha' : "SP"
            ,'Numeracao' : 5
            , 'Data_Emissao': datetime.datetime.now()
            } , 
            ignore_index=True)

In [ ]:
df.to_csv("atendimentos.csv")

In [ ]:
pd.read_csv('atendimentos.csv').tail()

In [ ]:
pd.to_datetime(ultimo_registro["Data_Emissao"]).values[0] < pd.to_datetime(inicio_expediente)

In [ ]:
data_emissao

In [ ]:
str(last_password).zfill(3)